Masterthesis

In [ ]:
# load modules
import random
import matplotlib.pyplot as plt

Set up an initial population of individuals with a predetermined number of chromosomes (ploidy_number) and loci per chromosomes (loci_number)

In [ ]:
def initialize_population(initial_size:int,ploidy_number:int,loci_number:int):
    """_summary_
    Initialize the starting population with a given number of individuals of the population (initial_size), with its ploidy level(ploidy_number)
    and number of loci per chromosome (loci_number)
    Args:
        initial_size (int): Size of the initial population of individuals to start with
        ploidy_number (int): Number of chromosome sets per individual
        loci_number (int): Number of loci per chromosome

    Returns:
        List of lists which represents the population with its individuals based on the given parameters
    """
    population = []
    for i in range(0, initial_size):
        individual = []
        for i in range(0, ploidy_number):
            chromosome_set = []
            for i in range(0, loci_number):
                chromosome_set.append(random.choice([0,1]))
            individual.append(chromosome_set)
        population.append(individual)
    return population

Cytoptype dynamics model that starts with an initial population of individuals that will study how the frequency of diploids, triploids and tetraploids will change after a certain number of generations. Per generation 2 by 2 individuals will be chosen to undergo meisosis and form a future generation.